<a href="https://colab.research.google.com/github/DAFNIKA/GUVI/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

In [ ]:
API_kEY="0ebcef01-70c7-4fc7-ab93-24a406b4fade"

url="https://api.harvardartmuseums.org/classification"

import requests
params={"apikey":API_kEY,
        "size":100}

response=requests.get(url,params)
response

In [ ]:
data=response.json()
type(data)

In [ ]:
data.keys()

In [ ]:
data['records']

In [ ]:
for i in data['records']:
  if i['objectcount'] >= 2500:
     print(i)

In [ ]:
url = "https://api.harvardartmuseums.org/object"
r = []
classifications = ["Vessels", "Prints", "Coins", "Paintings", "Drawings"]

for classification in classifications:
  for i in range(1, 26):
    params = {
        "apikey": API_kEY,
        "size": 100,
        "page": i,
        "classification": classification
    }
    response = requests.get(url, params)
    r.append(response)

r

In [ ]:
data = []
for response in r:
  data.append(response.json())
type(dict(data))

In [ ]:
for g in data:
  print(g.keys())


In [ ]:
for g in data:
   print(g['records'])

In [ ]:
for g in data:
  type(g['records'])

In [ ]:
for g in data:
   g['records'][1]

In [ ]:
metadata=[]
media=[]
colors=[]
for g in data:
 for i in g['records']:
  period_value = i.get('period')
  if isinstance(period_value, list) and period_value:
    period_value = str(period_value[0])
  elif isinstance(period_value, list):
    period_value = None
  elif period_value is not None:
    period_value = str(period_value)

  colorcount_value = i.get('colorcount')
  if isinstance(colorcount_value, list) and colorcount_value:
    try:
        colorcount_value = int(colorcount_value[0])
    except (ValueError, TypeError):
        colorcount_value = None
  elif isinstance(colorcount_value, list):
    colorcount_value = None
  elif colorcount_value is not None:
      try:
          colorcount_value = int(colorcount_value)
      except (ValueError, TypeError):
          colorcount_value = None


  metadata.append(dict(id=i.get('id'),
                       title=i.get('title'),
                      culture=i.get('culture'),
                       period=period_value,
                       century=i.get('century'),
                       medium=i.get('medium'),
                       dimensions=i.get('dimensions'),
                       description=i.get('description'),
                       department=i.get('department'),
                       classification=i.get('classification'),
                       accessionyear=i.get('accessionyear'),
                       accessionmethod=i.get('accessionmethod'),
                      ))
  media.append(dict(objectid=i.get('objectid'),
                       imagecount=i.get('imagecount'),
                      mediacount=i.get('mediacount'),
                       colorcount=colorcount_value,
                       rank=i.get('rank'),
                       datebegin=i.get('datebegin'),
                       dateend=i.get('dateend')
                      ))
  color_details=i.get('colors')
  if color_details:
    for j in color_details:
      colors.append(dict(objectid=i.get('objectid'),
                       color=j.get('color'),
                      spectrum=j.get('spectrum'),
                       hue=j.get('hue'),
                       percent=j.get('percent'),
                       css3=j.get('css3'),
                      ))

In [ ]:
metadata

In [ ]:
media

In [ ]:
colors

In [ ]:
import sqlite3

conn = sqlite3.connect("artifacts.db")
cur = conn.cursor()

cur.execute("""
CREATE TABLE IF NOT EXISTS artifact_metadata (
    id INTEGER PRIMARY KEY,
    title TEXT,
    culture TEXT,
    period TEXT,
    century TEXT,
    medium TEXT,
    dimensions TEXT,
    description TEXT,
    department TEXT,
    classification TEXT,
    accessionyear INTEGER,
    accessionmethod TEXT
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS artifact_media (
    objectid INTEGER,
    imagecount INTEGER,
    mediacount INTEGER,
    colorcount INTEGER,
    rank INTEGER,
    datebegin INTEGER,
    dateend INTEGER,
    FOREIGN KEY(objectid) REFERENCES artifact_metadata(id)
)
""")

cur.execute("""
CREATE TABLE IF NOT EXISTS artifact_colors (
    objectid INTEGER,
    color TEXT,
    spectrum TEXT,
    hue TEXT,
    percent REAL,
    css3 TEXT,
    FOREIGN KEY(objectid) REFERENCES artifact_metadata(id)
)
""")


cur.executemany("""
INSERT OR IGNORE INTO artifact_metadata (
    id, title, culture, period, century, medium, dimensions,
    description, department, classification, accessionyear, accessionmethod
) VALUES (
    :id, :title, :culture, :period, :century, :medium, :dimensions,
    :description, :department, :classification, :accessionyear, :accessionmethod
)
""", metadata)

cur.executemany("""
INSERT OR IGNORE INTO artifact_media (
    objectid, imagecount, mediacount, colorcount, rank, datebegin, dateend
) VALUES (
    :objectid, :imagecount, :mediacount, :colorcount, :rank, :datebegin, :dateend
)
""", media)

cur.executemany("""
INSERT OR IGNORE INTO artifact_colors (
    objectid, color, spectrum, hue, percent, css3
) VALUES (
    :objectid, :color, :spectrum, :hue, :percent, :css3
)
""", colors)

conn.commit()
conn.close()

print("✅ Tables created and data inserted successfully.")


In [ ]:
conn = sqlite3.connect("artifacts.db")
cur = conn.cursor()

cur.execute("SELECT COUNT(*) FROM artifact_metadata")
print("Number of records in artifact_metadata:", cur.fetchone()[0])

cur.execute("SELECT COUNT(*) FROM artifact_media")
print("Number of records in artifact_media:", cur.fetchone()[0])

cur.execute("SELECT COUNT(*) FROM artifact_colors")
print("Number of records in artifact_colors:", cur.fetchone()[0])

conn.close()


In [ ]:
import requests
import sqlite3

API_kEY = "0ebcef01-70c7-4fc7-ab93-24a406b4fade"
BASE_URL = "https://api.harvardartmuseums.org/object"
CLASSIFICATION = "Paintings"
RECORDS_TO_FETCH = 2500
PAGE_SIZE = 100


In [ ]:
metadata = []
media = []
colors = []

params = {
    "apikey": API_kEY,
    "classification": CLASSIFICATION,
    "size": PAGE_SIZE,
    "page": 1
}

while len(metadata) < RECORDS_TO_FETCH:
    response = requests.get(BASE_URL, params=params)
    data = response.json()

    for item in data.get("records", []):

        metadata.append({
            "id": item.get("id"),
            "title": item.get("title"),
            "culture": item.get("culture"),
            "period": item.get("period"),
            "century": item.get("century"),
            "medium": item.get("medium"),
            "dimensions": item.get("dimensions"),
            "description": item.get("description"),
            "department": item.get("division"),
            "classification": item.get("classification"),
            "accessionyear": item.get("accessionyear"),
            "accessionmethod": item.get("accessionmethod")
        })

        objectid = item.get("id")


        imagecount = len(item.get("images", []))
        mediacount = len(item.get("media", [])) if item.get("media") else 0
        colorcount = len(item.get("colors", [])) if item.get("colors") else 0

        media.append({
            "objectid": objectid,
            "imagecount": imagecount,
            "mediacount": mediacount,
            "colorcount": colorcount,
            "rank": item.get("rank"),
            "datebegin": item.get("datebegin"),
            "dateend": item.get("dateend")
        })


        for color in item.get("colors", []):
            colors.append({
                "objectid": objectid,
                "color": color.get("color"),
                "spectrum": color.get("spectrum"),
                "hue": color.get("hue"),
                "percent": color.get("percent"),
                "css3": color.get("css3")
            })

        if len(metadata) >= RECORDS_TO_FETCH:
            break

    if not data.get("info", {}).get("next"):
        break

    params["page"] += 1

print(f"Fetched {len(metadata)} metadata records")
print(f"Fetched {len(media)} media records")
print(f"Fetched {len(colors)} color records")


In [ ]:
conn = sqlite3.connect("artifacts.db")
cur = conn.cursor()


cur.executemany("""
INSERT OR IGNORE INTO artifact_metadata (
    id, title, culture, period, century, medium, dimensions,
    description, department, classification, accessionyear, accessionmethod
) VALUES (
    :id, :title, :culture, :period, :century, :medium, :dimensions,
    :description, :department, :classification, :accessionyear, :accessionmethod
)
""", metadata)


cur.executemany("""
INSERT OR IGNORE INTO artifact_media (
    objectid, imagecount, mediacount, colorcount, rank, datebegin, dateend
) VALUES (
    :objectid, :imagecount, :mediacount, :colorcount, :rank, :datebegin, :dateend
)
""", media)


cur.executemany("""
INSERT OR IGNORE INTO artifact_colors (
    objectid, color, spectrum, hue, percent, css3
) VALUES (
    :objectid, :color, :spectrum, :hue, :percent, :css3
)
""", colors)

conn.commit()
conn.close()

print("✅ Data inserted successfully!")


In [ ]:
import sqlite3

conn = sqlite3.connect("artifacts.db")
cur = conn.cursor()

cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print("Tables:", cur.fetchall())

cur.execute("SELECT COUNT(*) FROM artifact_metadata;")
count = cur.fetchone()[0]
print("Rows in artifact_metadata:", count)

conn.close()


In [ ]:
import sqlite3
import pandas as pd

def run_query(query):
    conn = sqlite3.connect('artifacts.db')
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df

query = """
SELECT * FROM artifact_metadata
WHERE century LIKE '%11th%'
AND culture LIKE '%Byzantine%'
LIMIT 10;
"""

df = run_query(query)
print(df)
